<h1>SD-WAN Basic Automation Bootcamp</h1>
<p>
The goal of this notebook is to get familar with working with the sd-wan apis
</p>
<ol>
    <li>Authenticate</li>
    <li>Get all devices in overlay</li>
    <li>Get all device templates</li>
    <li>Get Real Time BFD Sessions for a specic device</li>
</ol>
<h3>Import Libraries</h3>

In [15]:
"""
copyright Cisco
created by bshokrol
04/24/2021

"""
#ignore insecure requests warning
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#pip install requests
import requests
import getpass
import sys
import json
#pip install pandas
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 2000)

<h3>Create class</h3>

In [ ]:
class SdwanBootcamp:
    def __init__(self, vmanage_address: str, vmanage_username: str, vmanage_password: str, verify: bool = False):
        self.session = self.authenticate(vmanage_address, vmanage_username, vmanage_password)
        self.vmanage_address = vmanage_address
        self.verify = verify

    
    @staticmethod
    def authenticate(vmanage_address: str, vmanage_username: str, vmanage_password: str ,verify: bool = False) -> requests.session:
        session = requests.session()
        payload = {
            "j_username": vmanage_username,
            "j_password": vmanage_password
        }
        url = f"{vmanage_address}j_security_check"
        response = session.post(url=url, data=payload, verify=verify)
        if "<html>" in str(response.content):
            print("Authentication failed")
            sys.exit(0)
        print("Authenticated")
        session.headers["X-XSRF-TOKEN"] = session.get(f"{vmanage_address}dataservice/client/token").content
        session.headers["Content-Type"] = "application/json"
        return session

    def post_request(self, endpoint: str, payload: dict) -> requests.Response:
        data = json.dumps(payload)
        url = f"{self.vmanage_address}{endpoint}"
        return self.session.post(url=url, data=data, verify=self.verify)

    def get_request(self, endpoint: str) -> requests.Response:
        url = f"{self.vmanage_address}{endpoint}"
        return self.session.get(url=url, verify=self.verify)


<h3>1. Initialize class with credentials and Authenticate</h3>

In [12]:
vmanage_ip = "128.107.222.2"
vmanage_port = "8443"
vmanage_username = "admin"
vmanage_password = getpass.getpass("vmanage password: ")
vmanage_address = f"https://{vmanage_ip}:{vmanage_port}/"
sdwan_bootcamp = SdwanBootcamp(vmanage_address=vmanage_address, vmanage_username=vmanage_username, vmanage_password=vmanage_password)

vmanage password: ········
Authenticated


<h3>2. Get all devices in overlay using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-inventory">Get Connected Devices</a>

In [16]:
get_devices = "dataservice/device"
response = sdwan_bootcamp.get_request(endpoint=get_devices)
print(response.status_code)
devices_df = pd.DataFrame(response.json().get("data"))

200


In [17]:
devices_df

,deviceId,system-ip,host-name,reachability,status,personality,device-type,timezone,device-groups,lastupdated,domain-id,board-serial,certificate-validity,max-controllers,uuid,controlConnections,device-model,version,connectedVManages,site-id,latitude,longitude,isDeviceGeoData,platform,uptime-date,statusOrder,device-os,validity,state,state_description,model_sku,local-system-ip,total_cpu_count,testbed_mode,layoutLevel,ompPeers,linux_cpu_count,bfdSessionsUp,bfdSessions
0,1.1.1.11,1.1.1.11,vManage-1,reachable,normal,vmanage,vmanage,America/Los_Angeles,"[""No groups""]",1619285340641,0,01,Valid,0,3e36d6c7-ffe8-407e-a204-19f609627aad,3,vmanage,20.3.3,"[""1.1.1.11""]",1,37.666684,-122.777023,False,x86_64,1619284260000,4,next,valid,green,All daemons up,None,1.1.1.11,2,False,1,NaN,NaN,NaN,NaN
1,1.1.1.13,1.1.1.13,vSmart-1,reachable,normal,vsmart,vsmart,America/Los_Angeles,"[""No groups""]",1619285338807,1,03,Valid,NaN,2a31bd84-dbbf-44c1-9f1f-e99a2dde64c6,3,vsmart,20.3.3,"[""1.1.1.11""]",1,37.666684,-122.777023,False,x86_64,1619284020000,4,next,valid,green,All daemons up,None,1.1.1.13,1,False,2,2,NaN,NaN,NaN
2,1.1.1.12,1.1.1.12,vBond-1,reachable,normal,vbond,vbond,America/Los_Angeles,"[""No groups""]",1619284937648,NaN,02,Valid,NaN,6fb01aae-334a-470f-aa8e-0c4c97aa1ce6,NaN,vedge-cloud,20.3.3,"[""1.1.1.11""]",1,37.666684,-122.777023,False,x86_64,1619284020000,4,next,valid,green,All daemons up,None,1.1.1.12,2,False,3,NaN,1,NaN,NaN
3,1.1.1.15,1.1.1.15,vEdge-1,reachable,normal,vedge,vedge,America/Los_Angeles,"[""No groups""]",1619285345073,1,99773365,Valid,0,0f2d79af-d404-4fb4-9ce0-33cf49bf2e9f,2,vedge-cloud,20.3.3,"[""1.1.1.11""]",3,37.666684,-122.777023,False,x86_64,1619284980000,4,next,valid,green,All daemons up,None,1.1.1.15,2,False,4,1,1,1.0,1
4,1.1.1.14,1.1.1.14,vEdge-1,reachable,normal,vedge,vedge,America/Los_Angeles,"[""No groups""]",1619285311322,1,2E5C5084,Valid,0,60411df6-5dca-41d0-b4b3-73891d26e48e,2,vedge-cloud,20.3.3,"[""1.1.1.11""]",2,37.666684,-122.777023,False,x86_64,1619284980000,4,next,valid,green,All daemons up,None,1.1.1.14,2,False,4,1,1,1.0,1


<h3>3. Get all device templates in vmanage using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-configuration">Get Device Templates</a>


In [18]:
get_all_device_templates = "dataservice/template/device"
response = sdwan_bootcamp.get_request(get_all_device_templates)
print(response.status_code)
all_device_templates_df = pd.DataFrame(response.json().get("data"))

200


In [19]:
all_device_templates_df

,deviceType,lastUpdatedBy,templateClass,configType,templateId,factoryDefault,templateName,devicesAttached,templateDescription,lastUpdatedOn,templateAttached
0,vedge-ISR-4331,system,cedge,template,eefc7c43-cb38-4ff9-808a-ba21c6082205,True,Factory_Default_ISR_4331_V01,0,Factory Default template for ISR4331 device,1619284717304,14
1,vedge-C1111-8PLTELA,system,cedge,template,106f83a4-4d5e-4d95-9e88-4be6436b8924,True,Factory_Default_C1111_8PLTELA_V01,0,Factory Default template for C1111-8PLTELA,1619284719247,15
2,vedge-CSR-1000v,system,cedge,template,03b4efd4-4b3b-4edf-99d4-278970a61b0d,True,Default_AWS_TGW_CSR1000V_Template_V01,0,Default device template for AWS TGW CSR1000V,1619284720873,11
3,vedge-CSR-1000v,system,cedge,template,5d8a83f8-f4e0-444b-9ca6-a74020a28c12,True,Factory_Default_CSR_1000V_V01,0,Factory Default template for CSR 1000V,1619284722107,22
4,vedge-cloud,admin,vedge,template,602057d7-56cb-4f5a-a1df-9bba6647cc31,False,0eeacad9-4b2b-40d6-a1c6-049fb1c85d46,1,Device template of vEdge-1 with Site ID: 2 and...,1619285260846,12
5,vedge-cloud,admin,vedge,template,95d4f138-10c2-482c-9c8f-eb1f627cfcdb,False,6dfe49b7-d11c-4235-8b00-880eb7ff5c48,1,Device template of vEdge-1 with Site ID: 3 and...,1619285306600,12


<h3>Get current bfd session for a specific device using get_request</h3>
<a href="https://developer.cisco.com/docs/sdwan/#!device-realtime-monitoring/bfd">Real time monitoring BFD</a>


In [21]:
get_bfd_sess = "dataservice/device/bfd/history?deviceId={0}"
response = sdwan_bootcamp.get_request(get_bfd_sess.format("1.1.1.15"))
print(response.status_code)
bfd_sess_df = pd.DataFrame(response.json().get("data"))

200


In [22]:
bfd_sess_df

,dst-ip,src-ip,color,tx-pkts,vdevice-name,src-port,time-date,system-ip,index,dst-port,del,site-id,vdevice-host-name,vdevice-dataKey,proto,lastupdated,time,state,rx-pkts
0,10.1.1.5,10.2.1.5,default,0,1.1.1.15,12366,1619285260000,1.1.1.14,2,12366,0,2,vEdge-1,1.1.1.15-2-2021-04-24T10:27:40-0700,ipsec,1619295554782,2021-04-24T10:27:40-0700,down,0
1,10.1.1.5,10.2.1.5,default,10,1.1.1.15,12366,1619285261000,1.1.1.14,3,12366,0,2,vEdge-1,1.1.1.15-3-2021-04-24T10:27:41-0700,ipsec,1619295554782,2021-04-24T10:27:41-0700,up,10
2,10.1.1.5,10.2.1.5,default,66,1.1.1.15,12366,1619285296000,1.1.1.14,4,12366,0,2,vEdge-1,1.1.1.15-4-2021-04-24T10:28:16-0700,ipsec,1619295554782,2021-04-24T10:28:16-0700,down,58
3,10.1.1.5,10.2.1.5,default,30,1.1.1.15,12366,1619285308000,1.1.1.14,5,12366,0,2,vEdge-1,1.1.1.15-5-2021-04-24T10:28:28-0700,ipsec,1619295554782,2021-04-24T10:28:28-0700,up,19
4,10.1.1.5,10.2.1.5,default,110,1.1.1.15,12366,1619285327000,1.1.1.14,6,12366,0,2,vEdge-1,1.1.1.15-6-2021-04-24T10:28:47-0700,ipsec,1619295554782,2021-04-24T10:28:47-0700,down,102
5,10.1.1.5,10.2.1.5,default,30,1.1.1.15,12366,1619285327000,1.1.1.14,7,12366,1,2,vEdge-1,1.1.1.15-7-2021-04-24T10:28:47-0700,ipsec,1619295554782,2021-04-24T10:28:47-0700,down,19
6,10.1.1.5,10.2.1.5,default,0,1.1.1.15,12366,1619285327000,1.1.1.14,8,12366,0,2,vEdge-1,1.1.1.15-8-2021-04-24T10:28:47-0700,ipsec,1619295554782,2021-04-24T10:28:47-0700,down,0
7,10.1.1.5,10.2.1.5,default,7,1.1.1.15,12366,1619285329000,1.1.1.14,9,12366,0,2,vEdge-1,1.1.1.15-9-2021-04-24T10:28:49-0700,ipsec,1619295554782,2021-04-24T10:28:49-0700,up,6
